In [ ]:
from Bio import SeqIO
import re

In [ ]:
# Provides utility functions to find whether words like 'flu' are in viral
# genome sequence descriptions, copied from
# https://stackoverflow.com/a/5320179. Includes tests.

def find_whole_word(w, flags=re.IGNORECASE):
    return re.compile(r'\b({0})\b'.format(w), flags=flags).search

find_flu = find_whole_word('flu')
assert find_flu('flu') is not None
assert find_flu('aksdjflkaj') is None
assert find_flu('flue') is None
assert find_flu('flu|1234') is not None
assert find_flu('flu-like virus') is not None
assert find_flu('Human orthopneumovirus (HRSV) Subgroup B, complete genome') is None

find_influenza = find_whole_word('influenza')
assert find_influenza('influenza') is not None
assert find_influenza('Influenza A virus (A/Puerto Rico/8/1934(H1N1)) segment 1, complete sequence') is not None
assert find_influenza('Influenza B virus RNA 1, complete sequence') is not None
assert find_influenza('Influenza B virus (B/Lee/1940) segment 2, complete sequence.') is not None
assert find_influenza('Influenza C virus (C/Ann Arbor/1/50) PB2 gene for polymerase 2, complete cds.') is not None
assert find_influenza('Human parainfluenza virus 1, complete genome') is None
assert find_influenza('Human orthopneumovirus (HRSV) Subgroup B, complete genome') is None

find_human_immunodeficiency_virus = find_whole_word('human immunodeficiency virus')
assert find_human_immunodeficiency_virus('human immunodeficiency virus') is not None
assert find_human_immunodeficiency_virus('immunodeficiency virus') is None
assert find_human_immunodeficiency_virus('Human immunodeficiency virus 1, complete genome') is not None
assert find_human_immunodeficiency_virus('Simian immunodeficiency virus (SIV) - SIVagmMAL genomic RNA, complete genome, strain: ZMB') is None

find_hiv = find_whole_word('hiv')
assert find_hiv('hiv') is not None
assert find_hiv('chives') is None
assert find_hiv('hiv-1') is not None
assert find_hiv('Human immunodeficiency virus 1 (HIV-1)|1234') is not None
assert find_hiv('Simian immunodeficiency virus (SIV) - SIVagmMAL genomic RNA, complete genome, strain: ZMB') is None

In [ ]:
# Counts the number of unique {dna_window_size}-mers in all the viral
# genomes in the Reference Viral Database, a database attempting to
# comprehensively catalog viral genomes:
# https://hive.biochemistry.gwu.edu/rvdb.
#
# Notes:
# 1. You can download the dataset "U-RVDBv15.1.fasta" from the above
# website.
# 2. Running this chunk may require a machine with lots of RAM. I used an
# AWS EC2 m5a.12xlarge machine with 192 GB RAM.

dna_window_size = 42

# whether to exclude {dna_window_size}-mers exclusively in flu genomes
excl_influenza = True
# whether to exclude {dna_window_size}-mers exclusively in HIV genomes
excl_hiv = True

upper_bound_all_rvdb_windows = set() # stores {dna_window_size}-mers
num_seqs_seen = 0
seqs = SeqIO.parse("data/U-RVDBv15.1.fasta", "fasta")

# store counts of excluded viral genomes in each category
num_flu = 0
num_influenza = 0
num_human_immunodeficiency_virus = 0
num_hiv = 0

for seq in seqs:
    # report number of viral genomes screened so far
    num_seqs_seen += 1
    if num_seqs_seen % 100000 == 0:
        print(f'{num_seqs_seen} sequences screened')
    # exclude flu and HIV sequences, and count these
    excl = False
    if excl_influenza:
        if find_flu(seq.description) is not None:
            num_flu += 1
            excl = True
        if find_influenza(seq.description) is not None:
            num_influenza += 1
            excl = True
    if excl_hiv:
        if find_human_immunodeficiency_virus(seq.description) is not None:
            num_human_immunodeficiency_virus += 1
            excl = True
        if find_hiv(seq.description) is not None:
            num_hiv += 1
            excl = True
    if excl:
        continue
    # add all {dna_window_size}-mers in seq to set, including overlapping
    # ones
    sequence = str(seq.seq)
    for i in range(len(sequence)-(dna_window_size-1)):
        upper_bound_all_rvdb_windows.add(sequence[i:(i+dna_window_size)])

if excl_influenza:
    print(f'{num_flu} sequences with flu')
    print(f'{num_influenza} sequences with influenza')
if excl_hiv:
    print(f'''{num_human_immunodeficiency_virus} sequences with human
    immunodeficiency virus''')
    print(f'{num_hiv} sequences with hiv')
print(f'''{len(upper_bound_all_rvdb_windows)} unique {dna_window_size}-mers
      when excl_influenza = {excl_influenza}, excl_hiv = {excl_hiv}''')

In RVDBv15.1, there are

- 2 sequences with flu
- 661118 sequences with influenza
- 814283 sequences with human immunodeficiency virus
- 759673 sequences with hiv

There are **471495384 unique 39-mers** and **486339373 unique 42-mers** when `excl_influenza = True, excl_hiv = True`.